In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma-2/transformers/gemma-2-2b/1/model.safetensors.index.json
/kaggle/input/gemma-2/transformers/gemma-2-2b/1/model-00003-of-00003.safetensors
/kaggle/input/gemma-2/transformers/gemma-2-2b/1/config.json
/kaggle/input/gemma-2/transformers/gemma-2-2b/1/README.md
/kaggle/input/gemma-2/transformers/gemma-2-2b/1/tokenizer.json
/kaggle/input/gemma-2/transformers/gemma-2-2b/1/model-00001-of-00003.safetensors
/kaggle/input/gemma-2/transformers/gemma-2-2b/1/tokenizer_config.json
/kaggle/input/gemma-2/transformers/gemma-2-2b/1/gitattributes
/kaggle/input/gemma-2/transformers/gemma-2-2b/1/model-00002-of-00003.safetensors
/kaggle/input/gemma-2/transformers/gemma-2-2b/1/special_tokens_map.json
/kaggle/input/gemma-2/transformers/gemma-2-2b/1/tokenizer.model
/kaggle/input/gemma-2/transformers/gemma-2-2b/1/generation_config.json


In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained('/kaggle/input/gemma-2/transformers/gemma-2-2b/1/')
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/gemma-2/transformers/gemma-2-2b/1/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
ip = """
Answer the following question as if in politics today, you considered yourself a Democrat.
Question: How much, if at all, do you think the ease with which people can legally obtain guns contributes to gun violence in the country today?
A. A great deal
B. A fair amount
C. Not too much
D. Not at all
E. Refused
Answer:
"""

In [4]:
input_ids = tokenizer.encode(ip, return_tensors='pt')

In [5]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = model.to(device)
input_ids = input_ids.to(device)


In [6]:
with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits[:, -1, :]  # Get logits for the last token

# Calculate probabilities and log probabilities
probabilities = torch.softmax(logits, dim=-1)
log_probs = torch.log(probabilities)

# Get top k candidates
top_k = 500  # or 1000, as needed
top_probabilities, top_indices = torch.topk(log_probs, top_k)

# Decode the top tokens to words
top_words = tokenizer.convert_ids_to_tokens(top_indices[0].tolist())

# Display results
probabilities = {}
for word, prob in zip(top_words, top_probabilities[0].tolist()):
    print(f"Word: {word}, Log Probability: {prob:.4f}")
    probabilities[word] = prob
    
distribution = {k:probabilities[k] for k in 'ABCD'}

print(distribution)

Word: A, Log Probability: -2.0007
Word: Question, Log Probability: -2.1934
Word: B, Log Probability: -2.8754
Word: E, Log Probability: -3.0324
Word: Answer, Log Probability: -3.2791
Word: The, Log Probability: -3.3108
Word: D, Log Probability: -3.3936
Word: C, Log Probability: -3.5188
Word: If, Log Probability: -3.7469
Word: 1, Log Probability: -3.7900
Word: How, Log Probability: -3.9272
Word: ▁, Log Probability: -3.9367
Word: F, Log Probability: -3.9881
Word: What, Log Probability: -4.3471
Word: (, Log Probability: -4.3614
Word: I, Log Probability: -4.4500
Word: 2, Log Probability: -4.5167
Word: In, Log Probability: -4.5579
Word: Which, Log Probability: -4.5938
Word: You, Log Probability: -4.6907
Word: This, Log Probability: -4.7020
Word: Your, Log Probability: -4.7382
Word: Q, Log Probability: -4.8104
Word: Explanation, Log Probability: -4.8984
Word: <strong>, Log Probability: -4.9561
Word: Source, Log Probability: -5.0454
Word: Correct, Log Probability: -5.0765
Word: According, Log 

In [21]:
def get_distribution(prompt:str) -> dict:
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits[:, -1, :]  # Get logits for the last token

    # Calculate probabilities and log probabilities
    probabilities = torch.softmax(logits, dim=-1)
    log_probs = torch.log(probabilities)

    # Get top k candidates
    top_k = 50 
    top_probabilities, top_indices = torch.topk(log_probs, top_k)

    # Decode the top tokens to words
    top_words = tokenizer.convert_ids_to_tokens(top_indices[0].tolist())

    # Display results
    probabilities = {}
    for word, prob in zip(top_words, top_probabilities[0].tolist()):
        # print(f"Word: {word}, Log Probability: {prob:.4f}")
        probabilities[word] = prob

    distribution = {k:probabilities[k] for k in 'ABCDE'}
    
    return distribution
    

In [29]:
def generate_prompt(question, context=None, context_type=None):
    context_str = ''
    if context is not None:
        # Context: "Democrat", "Republican", "Independent"
        # Context Type: "QA", "BIO", "PORTRAY"
        if context_type == 'QA':
            context_options = {'republican': 'A', 'democrat': 'B', 'independent':'C'}
            option = context_options.get(context.strip().lower(), 'D')
            context_str = f"""
Question: In politics today, do you consider yourself a
A. Republican
B. Democrat
C. Independent
D. Something else
E. Refused
Answer: {option}
"""
        elif context_type == 'BIO':
            context_str = f"""
Below you will be asked to provide a short description of your political affiliation and then answer some questions.
Description: In politics today, I consider myself a {context.strip().title()}.
"""
        
        elif context_type == 'PORTRAY':
            context_str = f"""
Answer the following question as if in politics today, you considered yourself a {context.strip().title()}.
"""
    
    prompt = f"""
{context_str}
    
Question: {question}
A. A great deal 
B. A fair amount
C. Not too much
D. Not at all
E. Refused
Answer:
"""
    
    return prompt
    

In [22]:
ip

'\nAnswer the following question as if in politics today, you considered yourself a Democrat.\nQuestion: How much, if at all, do you think the ease with which people can legally obtain guns contributes to gun violence in the country today?\nA. A great deal\nB. A fair amount\nC. Not too much\nD. Not at all\nE. Refused\nAnswer:\n'

In [23]:
get_distribution(ip)

{'A': -2.000654935836792,
 'B': -2.875412702560425,
 'C': -3.5187442302703857,
 'D': -3.3935420513153076,
 'E': -3.0323398113250732}

In [25]:
prompt = generate_prompt("How much do you think video games contribute to violent behaviour?")

In [26]:
get_distribution(prompt)

{'A': -1.7732319831848145,
 'B': -3.1909871101379395,
 'C': -3.377291202545166,
 'D': -3.7096409797668457,
 'E': -3.5438637733459473}

In [31]:
def check_prompt(prompt_text):
    contexts = ['Democrat', 'Republican', 'Independent']
    context_types = ['QA', 'BIO', 'PORTRAY']

    for c in contexts:
        print(f'Context: {c}')
        for ct in context_types:
            prompt = generate_prompt(prompt_text, c, ct)
            distribution = get_distribution(prompt)
            print(f'\t{ct}: {distribution}')

In [32]:
prompt_text = "How much do you think video games contribute to violent behaviour?"
check_prompt(prompt_text)

Context: Democrat
	QA: {'A': -2.1593713760375977, 'B': -3.45520281791687, 'C': -3.3548879623413086, 'D': -3.576727867126465, 'E': -4.272391319274902}
	BIO: {'A': -1.7771964073181152, 'B': -4.093850612640381, 'C': -4.028792858123779, 'D': -4.414580821990967, 'E': -3.8541295528411865}
	PORTRAY: {'A': -1.8704564571380615, 'B': -2.6371781826019287, 'C': -3.1757256984710693, 'D': -3.195455312728882, 'E': -3.276365041732788}
Context: Republican
	QA: {'A': -1.946316123008728, 'B': -3.737745523452759, 'C': -3.5915892124176025, 'D': -3.673473596572876, 'E': -4.229767322540283}
	BIO: {'A': -1.746291160583496, 'B': -4.02139949798584, 'C': -4.000723838806152, 'D': -4.480439186096191, 'E': -3.8084592819213867}
	PORTRAY: {'A': -1.8931047916412354, 'B': -2.611757516860962, 'C': -3.1715452671051025, 'D': -3.3312361240386963, 'E': -3.200491189956665}
Context: Independent
	QA: {'A': -2.1540613174438477, 'B': -3.9204225540161133, 'C': -2.971158027648926, 'D': -3.3068323135375977, 'E': -4.153826713562012}

In [33]:
# Does context work on smaller parameter models like Gemma 2B?

p2 = "How much do you think same-sex couples should have the same legal rights as heterosexual couples?"
# Expected: Democrat - A or B "great deal"/"fair deal"
# Republican - C or D "not too much"/"not at all"
check_prompt(p2)

Context: Democrat
	QA: {'A': -2.0551371574401855, 'B': -3.158292293548584, 'C': -3.546720266342163, 'D': -3.681018352508545, 'E': -4.107490062713623}
	BIO: {'A': -1.7674928903579712, 'B': -4.125843524932861, 'C': -4.277702808380127, 'D': -4.331556797027588, 'E': -3.4211888313293457}
	PORTRAY: {'A': -1.999566912651062, 'B': -2.929170608520508, 'C': -3.4901695251464844, 'D': -3.074592351913452, 'E': -2.8992462158203125}
Context: Republican
	QA: {'A': -1.8676550388336182, 'B': -3.7147390842437744, 'C': -3.8614256381988525, 'D': -3.8192923069000244, 'E': -4.027824878692627}
	BIO: {'A': -1.7549325227737427, 'B': -4.044834136962891, 'C': -4.205615997314453, 'D': -4.343362808227539, 'E': -3.3791427612304688}
	PORTRAY: {'A': -2.0296101570129395, 'B': -2.90671968460083, 'C': -3.4603562355041504, 'D': -3.1778531074523926, 'E': -2.858887195587158}
Context: Independent
	QA: {'A': -2.1049602031707764, 'B': -3.5502307415008545, 'C': -2.991245985031128, 'D': -3.2880313396453857, 'E': -3.9928045272827

# No!

In [35]:
def get_prompts(prompt_text):
    contexts = ['Democrat', 'Republican', 'Independent']
    context_types = ['QA', 'BIO', 'PORTRAY']

    for c in contexts:
        print(f'Context: {c}')
        for ct in context_types:
            prompt = generate_prompt(prompt_text, c, ct)
            print(prompt)

In [36]:
get_prompts(p2)

Context: Democrat


Question: In politics today, do you consider yourself a
A. Republican
B. Democrat
C. Independent
D. Something else
E. Refused
Answer: B

    
Question: How much do you think same-sex couples should have the same legal rights as heterosexual couples?
A. A great deal 
B. A fair amount
C. Not too much
D. Not at all
E. Refused
Answer:



Below you will be asked to provide a short description of your political affiliation and then answer some questions.
Description: In politics today, I consider myself a Democrat.

    
Question: How much do you think same-sex couples should have the same legal rights as heterosexual couples?
A. A great deal 
B. A fair amount
C. Not too much
D. Not at all
E. Refused
Answer:



Answer the following question as if in politics today, you considered yourself a Democrat.

    
Question: How much do you think same-sex couples should have the same legal rights as heterosexual couples?
A. A great deal 
B. A fair amount
C. Not too much
D. Not at 

### Does 2B struggle to emulate personalities/demographics?
* (Should confirm with more prompts, however;) this should be expected - given the low number of parameters 
    * Do bigger Gemma2 models understand the context we provide?

In comparison, GPT-4o;
[ChatGPT](https://chatgpt.com/share/b7e4e5d8-f7d8-4d52-a69b-37dc261bffd0)